In [1]:
#include<iostream>
#include<vector>

In [2]:
namespace VoidPtrArray {
            typedef struct _name {
                _name() { p[0] = (void*)0x1; p[1] = (void*)0x2; p[2] = (void*)0x3; }
                void* p[3];
            } name;
        }

In [3]:
%%python

n = cppyy.gbl.VoidPtrArray.name()
assert n.p[0] == 0x1
assert n.p[1] == 0x2
assert n.p[2] == 0x3

In [4]:
namespace VectorConstCharStar {
            std::vector<const char*> test = {"hello"};
        }

In [5]:
%%python

import cppyy
ns = cppyy.gbl.VectorConstCharStar

assert len(ns.test) == 1
assert ns.test[0] == "hello"

ns.test.push_back("world")
assert len(ns.test) == 2
assert ns.test[0] == "hello"
assert ns.test[1] == "world"

In [6]:
namespace ArrayLike {
struct __attribute__((__packed__)) Vector3f {
    float x, y, z;
}; }


In [7]:
%%python

N = 5

v = cppyy.gbl.std.vector['ArrayLike::Vector3f'](N)

for i in range(N):
    d = v[i]
    d.x, d.y, d.z = i, i*N, i*N**2


In [8]:
%%python

for i in range(N):
    d = v[i]
    assert d.x == float(i)
    assert d.y == float(i*N)
    assert d.z == float(i*N**2)


#### STL-like class with preinc by-ref returns

In [10]:
namespace PreIncrement {
        struct Token {
            int value;
        };

struct Iterator {
    Token current;

    bool operator!=(const Iterator& rhs) {
        return rhs.current.value != current.value; }
    const Token& operator*() { return current; }
    Iterator& operator++() {
        current.value++;
        return *this;
    }
};

struct Stream {
    Iterator begin() { return Iterator(); }
    Iterator end() { return Iterator{10}; }
}; }

In [15]:
%%python

import cppyy
ns = cppyy.gbl.PreIncrement

stream = ns.Stream
stream.begin(stream)

Traceback (most recent call last):
  File "<string>", line 5, in <module>
TypeError: PreIncrement::Iterator PreIncrement::Stream::beginIterator PreIncrement::Stream::begin() =>
    TypeError: unbound method Stream::begin must be called with a Stream instance as first argument
